In [5]:
# Improve by ChatGPT & GitHub Copilot
# Running the requierements.ipynb
%run /bettik/PROJECTS/pr-data-ocean/riverama/Notebooks/requierements.ipynb

# Running functions_bar_igw.ipynb
%run /bettik/PROJECTS/pr-data-ocean/riverama/Notebooks/OSSE_borrador/IGW_BAR/functions_bar_igw.ipynb

In [6]:
def apply_mask(ds, mask):
    ssh_hf = ds.ssh_hf.where(mask == 1, np.nan)
    return ssh_hf.coarsen(x=4, y=4, boundary='trim').mean()

def fill_missing_data(ssh_hf):
    x_axis = Axis(ssh_hf.x.values, is_circle=True)
    y_axis = Axis(ssh_hf.y.values, is_circle=True)
    grid = Grid2D(y_axis, x_axis, ssh_hf.values.transpose(0, 1))
    has_converged, filled = fill.gauss_seidel(grid)
    ssh_hf_filled = ssh_hf.copy(deep=True, data=filled)
    return ssh_hf_filled

def create_cartesian_grid_data(ssh_hf_filled, dx):
    ENSLAT2D, ENSLON2D, i_lat, i_lon = create_cartesian_grid(ssh_hf_filled.lat[:, 0].values,
                                                             ssh_hf_filled.lon[0, :].values,
                                                             dx)
    return ENSLAT2D, ENSLON2D, i_lat, i_lon

def interpolate_to_cartesian(ssh_hf_filled, ENSLAT2D, ENSLON2D):
    points = np.column_stack((ssh_hf_filled.lat.data.flatten(), ssh_hf_filled.lon.data.flatten()))
    values = ssh_hf_filled.data.flatten()
    target_grid = np.column_stack((ENSLAT2D.flatten(), ENSLON2D.flatten()))

    array_cart_ssh = griddata(points, values, target_grid, method='linear')
    return array_cart_ssh.reshape(ENSLAT2D.shape)

def fill_cartesian_nans(array_cart_ssh, i_lat, i_lon):
    x_axis = Axis(np.arange(i_lon))
    y_axis = Axis(np.arange(i_lat))
    
    grid = Grid2D(y_axis, x_axis, array_cart_ssh)
    has_converged, filled = fill.gauss_seidel(grid)
    
    return filled

def extract_barotropic_tide(cart_ssh_hf, dx):
    return np.array(extract_bar_tide(cart_ssh_hf.values, dx))

def interpolate_back_to_original_grid(cart_ssh_filtered, ENSLAT2D, ENSLON2D, ssh_hf):
    lon2d, lat2d = np.meshgrid(ssh_hf.lon[0, :].values, ssh_hf.lat[:, 0].values)
    
    ssh_bar_array = np.array([griddata(
        np.array([ENSLAT2D.flatten(), ENSLON2D.flatten()]).T,
        cart_ssh_filtered.flatten(),
        (lat2d, lon2d),
        method='linear'
    )])
    
    ssh_bar = xr.DataArray(
        data=ssh_bar_array[0],
        dims=ssh_hf.dims,
        coords=ssh_hf.coords
    )
    
    return ssh_bar


def process_data(ds, mask, dx):
    ssh_hf = apply_mask(ds, mask)
    
    ssh_hf_filled = fill_missing_data(ssh_hf)
    ENSLAT2D, ENSLON2D, i_lat, i_lon = create_cartesian_grid_data(ssh_hf_filled, dx)
    
    array_cart_ssh = interpolate_to_cartesian(ssh_hf_filled, ENSLAT2D, ENSLON2D)
    filled = fill_cartesian_nans(array_cart_ssh, i_lat, i_lon)
    
    cart_ssh_hf = xr.DataArray(data=filled,
                               dims=["y", "x"],
                               coords=dict(y=(["y"], np.arange(i_lat)), x=(["x"], np.arange(i_lon))))
    
    cart_ssh_filtered = extract_barotropic_tide(cart_ssh_hf, dx)
    
    ssh_bar = interpolate_back_to_original_grid(cart_ssh_filtered, ENSLAT2D, ENSLON2D, ssh_hf)
    
    ssh_igw = ssh_hf_filled - ssh_bar
    
    return ssh_bar, ssh_igw

In [38]:
# Example usage
path_to_mask = "/bettik/PROJECTS/pr-data-ocean/riverama/Datos/CALEDO60/1_mesh_mask_TROPICO12_L125_tr21.nc"
path_to_input = "/bettik/PROJECTS/pr-data-ocean/riverama/Datos/Filtrage/ssh_hf/ssh_hf_"
path_to_save = "/bettik/PROJECTS/pr-data-ocean/riverama/Datos/Filtrage/ssh_bar/"
filename = "02.nc"
dx = 10

# Manually load the data
ssh_hf = xr.open_mfdataset(path_to_input + filename)
ds = ssh_hf.rename({'__xarray_dataarray_variable__': 'ssh_hf'}).sel(time_counter=pd.to_datetime('2014-02-02T11:30:00')).load()

# Load the mask
mask = xr.open_dataset(path_to_mask, drop_variables={"x", "y"}).tmaskutil[0, :, :].values

# Process the data
ssh_bar, ssh_igw = process_data(ds, mask, dx)


In [ ]:
doble_plot(ssh_bar, ssh_igw)

In [41]:
# Example usage
path_to_mask = "/bettik/PROJECTS/pr-data-ocean/riverama/Datos/CALEDO60/1_mesh_mask_TROPICO12_L125_tr21.nc"
path_to_input = "/bettik/PROJECTS/pr-data-ocean/riverama/Datos/Filtrage/ssh_hf/ssh_hf_"
path_to_save_bar = "/bettik/PROJECTS/pr-data-ocean/riverama/Datos/Filtrage/ssh_bar/"
path_to_save_igw = "/bettik/PROJECTS/pr-data-ocean/riverama/Datos/Filtrage/ssh_igw/"
filename = "02.nc"
dx = 10

# Manually load the data
ssh_hf = xr.open_mfdataset(path_to_input + filename)
ds = ssh_hf.rename({'__xarray_dataarray_variable__': 'ssh_hf'}).load()

# Load the mask
mask = xr.open_dataset(path_to_mask, drop_variables={"x", "y"}).tmaskutil[0, :, :].values

# Process the data for all time steps
ssh_bar_list = []
ssh_igw_list = []

for time_step in ds.time_counter:
    ds_time = ds.sel(time_counter=time_step)
    ssh_bar, ssh_igw = process_data(ds_time, mask, dx)
    ssh_bar_list.append(ssh_bar)
    ssh_igw_list.append(ssh_igw)

# Combine all the results into a single dataset
ssh_bar_combined = xr.concat(ssh_bar_list, dim='time_counter')
ssh_igw_combined = xr.concat(ssh_igw_list, dim='time_counter')

# Save the results
ssh_bar_combined.to_netcdf(path_to_save_bar + "ssh_bar_02.nc")
ssh_igw_combined.to_netcdf(path_to_save_igw + "ssh_igw_02.nc")

In [42]:
test_bar=xr.open_mfdataset("/bettik/PROJECTS/pr-data-ocean/riverama/Datos/Filtrage/ssh_bar/ssh_bar_02.nc")

In [43]:
test_bar

<xarray.Dataset> Size: 226MB
Dimensions:                        (y: 213, x: 197, time_counter: 672)
Coordinates:
    lat                            (y, x) float32 168kB dask.array<chunksize=(213, 197), meta=np.ndarray>
    lon                            (y, x) float32 168kB dask.array<chunksize=(213, 197), meta=np.ndarray>
  * time_counter                   (time_counter) datetime64[ns] 5kB 2014-02-...
Dimensions without coordinates: y, x
Data variables:
    __xarray_dataarray_variable__  (time_counter, y, x) float64 226MB dask.array<chunksize=(672, 213, 197), meta=np.ndarray>

In [7]:
# Example usage
path_to_mask = "/bettik/PROJECTS/pr-data-ocean/riverama/Datos/CALEDO60/1_mesh_mask_TROPICO12_L125_tr21.nc"
path_to_input = "/bettik/PROJECTS/pr-data-ocean/riverama/Datos/Filtrage/ssh_hf/ssh_hf_"
path_to_save_bar = "/bettik/PROJECTS/pr-data-ocean/riverama/Datos/Filtrage/ssh_bar/"
path_to_save_igw = "/bettik/PROJECTS/pr-data-ocean/riverama/Datos/Filtrage/ssh_igw/"
filename = "03.nc"
dx = 10

# Manually load the data
ssh_hf = xr.open_mfdataset(path_to_input + filename)
ds = ssh_hf.rename({'__xarray_dataarray_variable__': 'ssh_hf'}).load()

# Load the mask
mask = xr.open_dataset(path_to_mask, drop_variables={"x", "y"}).tmaskutil[0, :, :].values

# Process the data for all time steps using parallel processing
results = Parallel(n_jobs=-1)(delayed(process_data_for_time_step)(time_step, ds, mask, dx) for time_step in ds.time_counter)

# Separate the results
ssh_bar_list, ssh_igw_list = zip(*results)

# Combine all the results into a single dataset
ssh_bar_combined = xr.concat(ssh_bar_list, dim='time_counter')
ssh_igw_combined = xr.concat(ssh_igw_list, dim='time_counter')

# Extract the prefix from the filename
file_prefix = filename.split('.')[0]

# Save the results
ssh_bar_combined.to_netcdf(f"{path_to_save_bar}ssh_bar_{file_prefix}.nc")
ssh_igw_combined.to_netcdf(f"{path_to_save_igw}ssh_igw_{file_prefix}.nc")

NameError: name 'process_data_for_time_step' is not defined